In [1]:
import sys
import os

# Get the absolute path to the `src` directory
src_path = os.path.abspath(os.path.join('..', 'src'))

# Add the `src` directory to the Python path
if src_path not in sys.path:
    sys.path.append(src_path)

import numpy as np
import pandas as pd
from data_processing import fetch_data_up_to_last_week, fetch_last_week_data, preprocess_data, create_lstm_input
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from lstm_model import Attention

# Fetch and preprocess data
historical_data = fetch_data_up_to_last_week()
last_week_data = fetch_last_week_data()
preprocessed_historical_data, scaler = preprocess_data(historical_data)

# Prepare model input data
lookback = 20
X_input, _ = create_lstm_input(preprocessed_historical_data, target_column='Adj Close', lookback=lookback)

# Load the model
model = load_model("../models/lstm_tsla_model_rework_v5.h5", custom_objects={'Attention': Attention})

# Make predictions
predicted_prices = model.predict(X_input[-5:])

# Create a placeholder array to match the scaler's expected shape
predicted_prices_padded = np.zeros((predicted_prices.shape[0], scaler.min_.shape[0]))

# Set the predicted 'Adj Close' values in the first column (assuming 'Adj Close' was the first feature)
predicted_prices_padded[:, 0] = predicted_prices.flatten()

# Perform inverse scaling
predicted_prices_rescaled = scaler.inverse_transform(predicted_prices_padded)

# Extract only the 'Adj Close' column after inverse transformation
predicted_prices_rescaled = predicted_prices_rescaled[:, 0]

# Get actual prices for last week
actual_prices = last_week_data['Adj Close'].values

# Calculate error metrics
mape = mean_absolute_percentage_error(actual_prices, predicted_prices_rescaled)
rmse = np.sqrt(mean_squared_error(actual_prices, predicted_prices_rescaled))

# Display results
for i in range(len(actual_prices)):
    print(f"Day {i+1}: Predicted: {predicted_prices_rescaled[i]}, Actual: {actual_prices[i]}")

print(f"\nMean Absolute Percentage Error (MAPE): {mape * 100:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# Plot the results
plt.figure(figsize=(10,6))
plt.scatter(range(len(actual_prices)), actual_prices, label='Actual Prices', color='blue', marker='o')
plt.scatter(range(len(predicted_prices_rescaled)), predicted_prices_rescaled, label='Predicted Prices', color='red', marker='x')
plt.title('Predicted vs Actual Adjusted Close Prices for Last Week')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2024-10-11 15:10:46.879386: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-10-11 15:10:46.879406: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-10-11 15:10:46.879411: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-10-11 15:10:46.879427: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-11 15:10:46.879439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


2024-10-11 15:10:47.118047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


ValueError: non-broadcastable output operand with shape (5,1) doesn't match the broadcast shape (5,8)